In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score,train_test_split
from sklearn.cluster import KMeans
from __future__ import division
from gensim.models import Word2Vec
import re
from imblearn.over_sampling import SMOTE
#custom packages
from Twitch_Data.preprocessingtools import *
import custom_validation_tools

In [2]:
sentences = get_sentences('steming')

In [3]:
word_vec=Word2Vec(sentences)

In [4]:
def get_vectors(model):
    vectors=[]
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
    return vectors

In [5]:
vectors = get_vectors(word_vec)

In [6]:
_data =pd.read_json('Twitch_Data/AllLabeled.json')
_data.shape

(9939, 6)

In [7]:
_data2=pd.read_json('All_Labeled_NA.json')
_data2.head(1)

,class,edited message,features10,features100,features1000,features125,features15,features150,features20,features200,...,features4,features400,features5,features500,features6,features7,features8,features9,message,spam flag
0,1,@savjz what is the point of the quest in this ...,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[2.0, 0.0, 0.0, 1.0, 0.0, 0.0256410256, 0.02, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0256410256, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.02564102...","[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.025...","[2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, ...",@Savjz what is the point of the quest in this ...,0.712101


In [48]:
for n in [300]:
    cluster_model=KMeans(n_clusters=n)
    cluster_model.fit(vectors)
    _data2['features'+str(n)]=_data2.apply(lambda x:[extrace_advanced_features(word_vec,cluster_model,x['message'],x['edited message'],'steming')],axis=1)
    _data2['features'+str(n)]=_data2['features'+str(n)].apply(lambda x: x[0])

In [51]:
for n in [200,300,400,500,1000]:
    cluster_model=KMeans(n_clusters=n)
    cluster_model.fit(vectors)
    _data['features'+str(n)]=_data.apply(lambda x:[extrace_advanced_features(word_vec,cluster_model,x['message'],x['edited message'],'steming')],axis=1)
    _data['features'+str(n)]=_data['features'+str(n)].apply(lambda x: x[0])

In [52]:
_data2.to_json('All_Labeled_NA.json')
_data2.head(1)

,class,edited message,features10,features100,features15,features20,features25,features4,features5,features6,...,features9,message,spam flag,features125,features150,features200,features300,features400,features500,features1000
0,1,@savjz what is the point of the quest in this ...,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[2.0, 0.0, 0.0, 1.0, 0.0, 0.0256410256, 0.02, ...","[0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0256410256, 0...","[0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.02564102...",...,"[0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, ...",@Savjz what is the point of the quest in this ...,0.712101,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [8]:
cluster_model=KMeans(n_clusters=300)
cluster_model.fit(vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=300, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [10]:
Y=list(_data2['class'])
X=list(_data2['features300'])

In [11]:
model=RandomForestClassifier(n_estimators=300,max_depth=128)
model.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=128, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
importance = model.feature_importances_

In [19]:
def shrink_fearutes(n,X,importance):
    indices = sorted(range(100),key=lambda x:importance[x])[-n:]
    X_words=X[:100]
    X_tail=X[100:]
    new_words=[]
    for i in indices:
        new_words.append(X[i])
    return list(new_words)+list(X_tail)

In [21]:
spam = _data2[_data2['class']==0].shape[0]
total = _data2.shape[0]
spam/float(total)

0.739461655662773

In [55]:
splitter = StratifiedShuffleSplit(n_splits=5,test_size=.3)
trees = 600
tree_height = 128
model=RandomForestClassifier(n_estimators=trees,max_depth=tree_height)
Y=list(_data2['class'])
for n in [300,400,500]:
    X = list(_data2['features'+str(n)])
    print 'for ' +str(n)+' clusters and for trees of height'+str(tree_height)+' and there being '+str(trees)+' trees'
    #print 'Training accucary: '+str(Train_Accucary(model,X,Y,splitter.split(X,Y)))
    print 'CV accucary: '+str(cross_val_score(model,X=X,y=Y,cv=splitter.split(X,Y)).mean())
    #print 'recall Score: '+str(cross_val_score(model,X=X_clusters,y=Y,cv=splitter.split(X_clusters,Y),scoring='recall').mean())
    print 'F1 Score: '+str(cross_val_score(model,X=X,y=Y,cv=splitter.split(X,Y),scoring='f1').mean())

for 300 clusters and for trees of height128 and there being 600 trees
CV accucary: 0.853491324587
F1 Score: 0.675655515486
for 400 clusters and for trees of height128 and there being 600 trees
CV accucary: 0.851713922979
F1 Score: 0.678071849321
for 500 clusters and for trees of height128 and there being 600 trees
CV accucary: 0.85129073212
F1 Score: 0.686187108814


In [56]:

_data.head(1)

,auto_flag,class,edited message,features100,message,spam flag,features200,features300,features400,features500,features1000
0,0,1,@savjz what is the point of the quest in this ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",@Savjz what is the point of the quest in this ...,0.712101,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [57]:
model=RandomForestClassifier(n_estimators=300,max_depth=128)
splitter = StratifiedShuffleSplit(n_splits=5,test_size=.3)
X=list(_data['features300'])
Y=list(_data['class'])
flag=list(_data['auto_flag'])

In [33]:
reload(custom_validation_tools)

<module 'custom_validation_tools' from 'custom_validation_tools.py'>

In [58]:
custom_validation_tools.top_guess_accucary(splitter.split(X,Y),model,X,Y,flag)

0.9308290610488419

In [5]:
X=list(_data2['features300'])
Y=list(_data2['class'])
model=RandomForestClassifier(n_estimators=300,max_depth=128)
model.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=128, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
with open('current_model','w+') as file:
    pickle.dump(model,file)

In [13]:
with open('current_wv_model','w+') as file:
    pickle.dump(word_vec,file)

In [14]:
with open('current_cluster_model','w+') as file:
    pickle.dump(cluster_model,file)